# Visualising Data in Maps

Another very useful way to visualize data is to plot its location based on a map and maybe color its surroundings given certain information about the data being displayed.

## Latitude and Longitude

Latitude and longitude are like the x and y coordinates for earth. Unlike most cartesian plots earth is not exactly flat thus we adapt a different system (lattitude and longitude) which allows us to describe the earth more accurately given its elipsoidal shape. Every address, location and point on the planet has a unique lat, long pair which describe its location. Lets give it a go at plotting somethings on a map !

In [1]:
# The map visualisation library we will be using
import folium
import pandas as pd
import json
from dds_lab.datasets import hotel, hood
from dds_lab.map_render import *

In [2]:
allotments_csv = pd.read_csv(hood + "/allotments.csv")
lat_long = allotments_csv["Location"]

# Set up folium map centered in the meadows
fmap = folium.Map(location=[55.944, -3.192],zoom_start=12)
for i, x in enumerate(lat_long):
    lat, long = list(map(float, x.split(",")))
    # Plot marker with information
    fmap.simple_marker([lat, long],
                      popup="Number of plots: " + str(allotments_csv["Total plots"][i]))

# Click away on the markers and find out how many available places
# a given alotment has 
inline_map(fmap)

Now the downside here is that whilst we get to where the allotments can be found in edinburgh and when we click on them we know how many plots an area has we dont get an overall feel of the numebrs without clicking on every marker. This is where a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) can be a usefull visualization.

## Choropleth

A Choropleth is a map which is colored on a number that changes per area. This may be percentage employed per goverment constituency or number of pupils per school catchment area. In the following snippet we aim to do a Choropleth which is colored based on the number of rooms available per hotel catchement area. The catchment areas of the hotels were estimated with a procedure called [Voronoi Tesselation](https://en.wikipedia.org/wiki/Voronoi_diagram) which estimates which areas are closer to one hotel than another and draws a polygon like boundary for every hotel:

In [ ]:
vor_path = hotel + "/vor.json"

# Open room data
data = pd.read_csv(hotel + "/hotels.csv")
# Remove hotels (rows) with no information about rooms
data = data.dropna(subset=["ROOMS"], how="all")
# data = data.fillna(0)
# Cast number of rooms to integer
data["ROOMS"] = data["ROOMS"].astype(int)

# Create map using folium and center view in edinburgh meadows
fmap = folium.Map(location=[55.944, -3.192],zoom_start=13)

# Plot cloropleth with catchment areas for hotels
# The catchment areas were estimated using a voronoi tessalation
fmap.geo_json(geo_path='vor.json',
              data=data,
              columns=["HOTEL", "ROOMS"],
              key_on="feature.id",
              fill_color="PuRd", fill_opacity=0.5, line_opacity=0.4,
              legend_name="Number of rooms")

# plot centroids
hotel_dict = json.loads(open(hotel + "/hotel.json",'r').read())["features"]
for i, x in enumerate(hotel_dict):
    fmap.polygon_marker(x["geometry"]["coordinates"][::-1],
                       radius=2,
                       popup="Hotel name:" + x["properties"]["hotel"])


embed_map(fmap)